# Business Questions — Evidence Notebook (Gold Layer)

This notebook provides **reproducible analytical evidence** for the MVP business questions using the Gold star schema.

**Scope:** Jan/2023 to Dec/2024 (`AnoMes` 202301–202412)  
**Source:** `mvp_pix.gold` (fact + dimensions + helper view)

Notes:
- Each answered question produces a single primary output designed for screenshots.
- Two business questions (income level; essential vs non-essential expenses) are intentionally not answered due to dataset limitations and are discussed in the final conclusion.

In [0]:
from pyspark.sql import functions as F

SCOPE_START = "202301"
SCOPE_END   = "202412"

SNAPSHOT_MONTH = "202412"  # used to generate chart-friendly, screenshot-ready outputs

fact = spark.table("mvp_pix.gold.fato_transacoes_pix")
t    = spark.table("mvp_pix.gold.dim_tempo")
u    = spark.table("mvp_pix.gold.dim_usuario")
r    = spark.table("mvp_pix.gold.dim_regiao")
n    = spark.table("mvp_pix.gold.dim_natureza")
p    = spark.table("mvp_pix.gold.dim_finalidade")
m    = spark.table("mvp_pix.gold.dim_forma_iniciacao")

vw_conc = spark.table("mvp_pix.gold.vw_regional_concentration")


## Q1 — Monthly evolution + top payer age groups
**Question:** Which age groups most frequently make PIX payments, and how does this pattern evolve over time?

Evidence is provided in two outputs:
1) Monthly evolution of total PIX activity (baseline trend).
2) Top payer age groups in the 2023–2024 scope (to identify the most active profiles).

In [0]:
%sql
SELECT
  t.AnoMes,
  SUM(f.quantidade_transacoes) AS total_transactions,
  SUM(f.valor_total)           AS total_value_brl
FROM mvp_pix.gold.fato_transacoes_pix f
JOIN mvp_pix.gold.dim_tempo t
  ON f.id_tempo = t.id_tempo
WHERE t.AnoMes BETWEEN '202301' AND '202412'
GROUP BY t.AnoMes
ORDER BY t.AnoMes;


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
  up.faixa_etaria AS payer_age_group,
  SUM(f.quantidade_transacoes) AS total_transactions,
  SUM(f.valor_total)           AS total_value_brl
FROM mvp_pix.gold.fato_transacoes_pix f
JOIN mvp_pix.gold.dim_tempo   t  ON f.id_tempo           = t.id_tempo
JOIN mvp_pix.gold.dim_usuario up ON f.id_usuario_pagador = up.id_usuario
WHERE t.AnoMes BETWEEN '202301' AND '202412'
GROUP BY up.faixa_etaria
ORDER BY total_transactions DESC;


## Q2 — Payer vs receiver age-group interaction
**Question:** Are there relevant differences between payers and receivers in terms of age distribution and transaction volume?

Primary evidence: a ranked table of payer → receiver age-group combinations (top interactions).


In [0]:
%sql
SELECT
  up.faixa_etaria AS payer_age_group,
  ur.faixa_etaria AS receiver_age_group,
  SUM(f.quantidade_transacoes) AS total_transactions,
  SUM(f.valor_total)           AS total_value_brl
FROM mvp_pix.gold.fato_transacoes_pix f
JOIN mvp_pix.gold.dim_usuario up ON f.id_usuario_pagador   = up.id_usuario
JOIN mvp_pix.gold.dim_usuario ur ON f.id_usuario_recebedor = ur.id_usuario
JOIN mvp_pix.gold.dim_tempo   t  ON f.id_tempo            = t.id_tempo
WHERE t.AnoMes BETWEEN '202301' AND '202412'
GROUP BY up.faixa_etaria, ur.faixa_etaria
ORDER BY total_transactions DESC
LIMIT 25;


## Q3 — Purpose by age group (payer perspective)
**Question:** How does transaction purpose vary across different age groups?

Primary evidence: purpose distribution by payer age group (by transaction count).


In [0]:
%sql
WITH base AS (
  SELECT
    u.faixa_etaria AS payer_age_group,
    p.finalidade   AS purpose,
    SUM(f.quantidade_transacoes) AS total_transactions
  FROM mvp_pix.gold.fato_transacoes_pix f
  JOIN mvp_pix.gold.dim_tempo      t ON f.id_tempo           = t.id_tempo
  JOIN mvp_pix.gold.dim_usuario    u ON f.id_usuario_pagador = u.id_usuario
  JOIN mvp_pix.gold.dim_finalidade p ON f.id_finalidade      = p.id_finalidade
  WHERE t.AnoMes = '202412'
  GROUP BY payer_age_group, purpose
),
top_purposes AS (
  SELECT purpose
  FROM base
  GROUP BY purpose
  ORDER BY SUM(total_transactions) DESC
  LIMIT 6
),
filtered AS (
  SELECT
    payer_age_group,
    CASE WHEN purpose IN (SELECT purpose FROM top_purposes) THEN purpose ELSE 'Other' END AS purpose_group,
    SUM(total_transactions) AS total_transactions
  FROM base
  GROUP BY payer_age_group, purpose_group
)
SELECT *
FROM filtered
ORDER BY payer_age_group, total_transactions DESC;


Databricks visualization. Run in Databricks to view.

## Q4 — Regional differences (purpose and nature)
**Question:** Are there regional differences in PIX usage considering transaction purpose, nature, and volume?

Primary evidence: payer region × nature (by total value).


In [0]:
%sql
WITH base AS (
  SELECT
    rp.regiao  AS payer_region,
    n.natureza AS nature,
    SUM(f.valor_total) AS total_value_brl
  FROM mvp_pix.gold.fato_transacoes_pix f
  JOIN mvp_pix.gold.dim_tempo    t  ON f.id_tempo          = t.id_tempo
  JOIN mvp_pix.gold.dim_regiao   rp ON f.id_regiao_pagador = rp.id_regiao
  JOIN mvp_pix.gold.dim_natureza n  ON f.id_natureza       = n.id_natureza
  WHERE t.AnoMes = '202412'
  GROUP BY payer_region, nature
),
ranked AS (
  SELECT
    *,
    DENSE_RANK() OVER (PARTITION BY payer_region ORDER BY total_value_brl DESC) AS rnk
  FROM base
)
SELECT payer_region, nature, total_value_brl
FROM ranked
WHERE rnk <= 4
ORDER BY payer_region, total_value_brl DESC;


Databricks visualization. Run in Databricks to view.

## Q5 — Most common patterns (age × nature × purpose)
**Question:** Which combinations of age group, transaction nature, and transaction purpose represent the most common PIX usage patterns?

Primary evidence: Top patterns by transaction count (payer perspective).


In [0]:
%sql
SELECT
  u.faixa_etaria AS payer_age_group,
  n.natureza     AS nature,
  p.finalidade   AS purpose,
  SUM(f.quantidade_transacoes) AS total_transactions,
  SUM(f.valor_total)           AS total_value_brl
FROM mvp_pix.gold.fato_transacoes_pix f
JOIN mvp_pix.gold.dim_tempo      t ON f.id_tempo           = t.id_tempo
JOIN mvp_pix.gold.dim_usuario    u ON f.id_usuario_pagador = u.id_usuario
JOIN mvp_pix.gold.dim_natureza   n ON f.id_natureza        = n.id_natureza
JOIN mvp_pix.gold.dim_finalidade p ON f.id_finalidade      = p.id_finalidade
WHERE t.AnoMes BETWEEN '202301' AND '202412'
GROUP BY payer_age_group, nature, purpose
ORDER BY total_transactions DESC
LIMIT 30;


## Q6 — Payer vs receiver profiles across regions and age groups
**Question:** How does the distribution of PIX usage differ between payer and receiver profiles across regions and age groups?

Primary evidence: role-playing comparison (payer vs receiver) by region and age group.


In [0]:
%sql
WITH payer AS (
  SELECT
    rp.regiao AS region,
    up.faixa_etaria AS age_group,
    SUM(f.quantidade_transacoes) AS total_transactions
  FROM mvp_pix.gold.fato_transacoes_pix f
  JOIN mvp_pix.gold.dim_tempo   t  ON f.id_tempo           = t.id_tempo
  JOIN mvp_pix.gold.dim_regiao  rp ON f.id_regiao_pagador  = rp.id_regiao
  JOIN mvp_pix.gold.dim_usuario up ON f.id_usuario_pagador = up.id_usuario
  WHERE t.AnoMes = '202412'
  GROUP BY region, age_group
),
receiver AS (
  SELECT
    rr.regiao AS region,
    ur.faixa_etaria AS age_group,
    SUM(f.quantidade_transacoes) AS total_transactions
  FROM mvp_pix.gold.fato_transacoes_pix f
  JOIN mvp_pix.gold.dim_tempo   t  ON f.id_tempo            = t.id_tempo
  JOIN mvp_pix.gold.dim_regiao  rr ON f.id_regiao_recebedor = rr.id_regiao
  JOIN mvp_pix.gold.dim_usuario ur ON f.id_usuario_recebedor= ur.id_usuario
  WHERE t.AnoMes = '202412'
  GROUP BY region, age_group
),
unioned AS (
  SELECT region, age_group, 'Payer' AS role, total_transactions FROM payer
  UNION ALL
  SELECT region, age_group, 'Receiver' AS role, total_transactions FROM receiver
)
SELECT region, role, total_transactions
FROM unioned
WHERE age_group = '30–39'
ORDER BY region, role;


Databricks visualization. Run in Databricks to view.

## Q7 — Regional concentration (value vs transaction count)
**Question:** How concentrated is PIX usage across regions when comparing total transaction value and transaction count?

Primary evidence uses the Gold helper view:
`mvp_pix.gold.vw_regional_concentration`


In [0]:
%sql
SELECT
  AnoMes,
  papel   AS role_pt,
  regiao  AS region,
  share_valor             AS value_share,
  share_quantidade        AS transaction_share,
  rank_valor              AS value_rank,
  rank_quantidade         AS transaction_rank,
  cumulative_share_valor  AS cumulative_value_share,
  cumulative_share_quantidade AS cumulative_transaction_share
FROM mvp_pix.gold.vw_regional_concentration
WHERE AnoMes BETWEEN '202301' AND '202412'
ORDER BY AnoMes, role_pt, value_rank;


Databricks visualization. Run in Databricks to view.

## Notes

This notebook focuses on evidence generation for the business questions that are answerable with the available aggregated PIX dataset (2023–2024).

Two proposed questions are not answered here and are discussed in the final conclusion:
- income level of the most active users
- essential vs non-essential expense classification
